In [1]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
5,application_1603723084707_0006,pyspark,idle,Link,Link,
7,application_1603723084707_0008,pyspark,idle,Link,Link,


In [2]:
#importing necessary libaries
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StringType
from pyspark import SQLContext
from itertools import islice
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1603723084707_0009,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
import json

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#creating the context
sqlContext = SQLContext(sc)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
original_books = sc.textFile("s3n://analytics-50043/data/meta_Kindle_Store.json").map(lambda line: eval(line))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
original_books.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'asin': '1603420304', 'description': "In less time and for less money than it takes to order pizza, you can make it yourself!Three harried but heatlh-conscious college students compiled and tested this collection of more than 200 tasty, hearty, inexpensive recipes anyone can cook -- yes, anyone!Whether you're short on cash, fearful of fat, counting your calories, or just miss home cooking, The Healthy College Cookbook offers everything you need to make good food yourself.", 'price': 7.69, 'imUrl': 'http://ecx.images-amazon.com/images/I/51IEqPrF%2B9L._BO2,204,203,200_PIsitb-sticker-v3-big,TopRight,0,-55_SX278_SY278_PIkin4,BottomRight,1,22_AA300_SH20_OU01_.jpg', 'related': {'also_viewed': ['B001OLRKLQ', 'B004J35JIC', 'B00505UP8M', 'B004GTLKEQ', 'B005KWMS8U', 'B00BS03TYU', 'B001MT5NXW', 'B00A86JE3K', 'B00D694Y9U', 'B00DSVUVXY', 'B008EN3W6Y', 'B00BS03W5Q', 'B008161J1O', 'B0089LOJH2', 'B00ENSBJYQ', 'B00C7C040U', 'B00DH410VY', 'B00CMVFW4O', 'B00C89GS1Q', 'B0035FZJ9Y', 'B004GTLFUK', 'B00H24W

In [19]:
original_books.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

434702

In [20]:
import re
def clean_string(string):
    if not isinstance(string, str):
        return string
    string = re.sub('<[^<]+?>', '', string)
    return html.unescape(string).strip()

def get_desc(dictionary):
    listOfList = list(dictionary.values())
    flatList = [ item for elem in listOfList for item in elem]
    return list(set(flatList))


def get_cat(cat_list):
    flatList = [cate for subcate in cat_list for cate in subcate]
    if len(cat_list) == 0 or len(cat_list[0]) == 0:
        return ""
    if "Kindle eBooks" in flatList:
        kindle_ebook_index = flatList.index("Kindle eBooks")
        if kindle_ebook_index < len(flatList) - 1:
            return clean_string(flatList[kindle_ebook_index + 1])
    elif "Books" in flatList:
        book_index = flatList.index("Books")
        if book_index < len(flatList) - 1:
            return clean_string(flatList[book_index + 1])
    return ""
    
get_desc({"a": [1,2,3,4,5], "b":[2,3,4,1,6,7,5]})
get_cat([['Books', 'Biographies & Memoirs', 'Historical'], ['Books', 'Biographies & Memoirs', 'Leaders & Notable People', 'Presidents & Heads of State'], 
         ['Books', 'Biographies & Memoirs', 'Memoirs'], ['Books', 'Literature & Fiction', 'Classics'], ['Kindle Store', 'Kindle eBooks', 'Biographies & Memoirs', 'Historical'], 
         ['Kindle Store', 'Kindle eBooks', 'Biographies & Memoirs', 'Leaders & Notable People', 'Presidents & Heads of State'], 
         ['Kindle Store', 'Kindle eBooks', 'Biographies & Memoirs', 'Memoirs'], ['Kindle Store', 'Kindle eBooks', 'Literature & Fiction', 'Classics', 'Historical']])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Biographies & Memoirs'

In [21]:
books = original_books.map(lambda line: {"asin": line['asin'], "price": line.get("price", ""), "imUrl": line.get("imUrl"), 
                                         "title_prof": clean_string(line.get("title", "")), "author": clean_string(line.get("brand", "")), "cate": get_cat(line.get("categories", [[]])),
                                         "description": clean_string(line.get("description", "")), "related": get_desc(line.get("related", {}))})
books.count()
books.take(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[{'asin': '1603420304', 'price': 7.69, 'imUrl': 'http://ecx.images-amazon.com/images/I/51IEqPrF%2B9L._BO2,204,203,200_PIsitb-sticker-v3-big,TopRight,0,-55_SX278_SY278_PIkin4,BottomRight,1,22_AA300_SH20_OU01_.jpg', 'title_prof': '', 'author': '', 'cate': 'Cookbooks, Food & Wine', 'description': "In less time and for less money than it takes to order pizza, you can make it yourself!Three harried but heatlh-conscious college students compiled and tested this collection of more than 200 tasty, hearty, inexpensive recipes anyone can cook -- yes, anyone!Whether you're short on cash, fearful of fat, counting your calories, or just miss home cooking, The Healthy College Cookbook offers everything you need to make good food yourself.", 'related': ['B00JEOMV1E', 'B00A86JE3K', 'B004GTLKEQ', 'B00BS03W5Q', 'B008161J1O', 'B00DSVUVXY', 'B0089LOJH2', 'B001OLRKLQ', 'B00505UP8M', 'B00H24WT2E', 'B00DH410VY', 'B005KWMS8U', 'B00BS03TYU', 'B00CVS44OW', 'B004GTLFUK', 'B00C7C040U', 'B004J35JIC', 'B008EN3W6Y',

In [26]:
fields =['asin','price','imUrl',"title_prof","author","cate",'description','related']

schema = StructType([
StructField(field,StringType(),True) for field in fields
])


#rdd = sc.parallelize(js)
profDF = spark.createDataFrame(books,schema)

profDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|      asin|price|               imUrl|          title_prof|author|                cate|         description|             related|
+----------+-----+--------------------+--------------------+------+--------------------+--------------------+--------------------+
|1603420304| 7.69|http://ecx.images...|                    |      |Cookbooks, Food &...|In less time and ...|[B00JEOMV1E, B00A...|
|B0002IQ15S|19.99|http://ecx.images...|Mobility IGO AUTO...|      |                    |This universal DC...|[B00EOE6COQ, B001...|
|B000F83SZQ|  0.0|http://ecx.images...|                    |      |Mystery, Thriller...|                    |[B00KL13WXK, B00L...|
|B000F83TEQ|     |http://ecx.images...|                    |      |Mystery, Thriller...|                    |[B00IS80K60, B00D...|
|B000F83STC|     |http://g-ecx.imag...|                    |      |Literature & Fic

In [27]:
categories = profDF.select('cate').distinct()
categories.show(150, False)
categories.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------------------+
|cate                        |
+----------------------------+
|Parenting & Relationships   |
|Politics & Social Sciences  |
|Biographies & Memoirs       |
|Gay & Lesbian               |
|Cookbooks, Food & Wine      |
|Romance                     |
|Nonfiction                  |
|Education & Teaching        |
|History                     |
|Self-Help                   |
|                            |
|Health, Fitness & Dieting   |
|Comics & Graphic Novels     |
|Travel                      |
|Teen & Young Adult          |
|Sports & Outdoors           |
|Foreign Languages           |
|Science & Math              |
|Reference                   |
|Science Fiction & Fantasy   |
|Business & Money            |
|Crafts, Hobbies & Home      |
|Literature & Fiction        |
|Computers & Technology      |
|Children's eBooks           |
|Mystery, Thriller & Suspense|
|Professional & Technical    |
|Religion & Spirituality     |
|Arts & Photography          |
|Humor &

In [5]:
original_books_ori = sc.textFile("s3n://analytics-50043/data/meta_Kindle_Store_original.json")\
                    .map(lambda line: json.loads(line)).filter(lambda line: line['title'] != '')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:

import re
import html

def clean_string(string):
    string = re.sub('<[^<]+?>', '', string)
    return html.unescape(string).strip()

def clean_rank(rank_string):
    if not isinstance(rank_string, str):
        return rank_string
    return clean_string(rank_string.split(' ')[0].replace(',', ''))

def clean_author(brand_string):
    if not isinstance(brand_string, str):
        return ""
    if (len(brand_string) > 20 and brand_string[:14].lower() == "visit amazon's" and brand_string[-4:].lower() == 'page'):
        return clean_string(brand_string[15: -5])
    return clean_string(brand_string)
    
def merge_two_list(list1, list2):
    list1.extend(x for x in list2 if x not in list1)
    return list1

def get_single_cate(cate_list):
    return '' if len(cate_list) < 3 else clean_string(cate_list[2])

def clean_title(string):
    # not seperated
    if len(string) >= 250 and len(string.split(',')) >= 4:
        string = string.split(',')[2]
    return clean_string(string.replace(' - Kindle edition', ''))

print(clean_title('By Force of Instinct: A Pride &amp; Prejudice Variation - Kindle edition'))

print(clean_rank('1,716,849 Paid in Kindle Store ('))
print(clean_author("Visit Amazon's Paul A. Craig Page"))
(print(merge_two_list(['1','3','5'],['5','3', 7,8,'8'])))
print(clean_string('<html>i am a title &amp; I am mom&#39;s good boy</html>'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

By Force of Instinct: A Pride & Prejudice Variation
1716849
Paul A. Craig
['1', '3', '5', 7, 8, '8']
i am a title & I am mom's good boy

In [7]:
books_ori = original_books_ori.map(lambda line: {'asin': line['asin'], 'category': get_single_cate(line['category']), 'title': clean_title(line['title']), 
                                         'author': clean_author(line['brand']), 'rank': clean_rank(line['rank']),
                                      'recommendation': line['also_view']})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
books_ori.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

488910

In [12]:
books_ori.first()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'asin': 'B00128EFK0', 'category': 'Religion & Spirituality', 'title': "Approaching the Holy-A Season in God's House", 'author': 'Dennis G. Crump', 'rank': '4646191', 'recommendation': []}

In [13]:
books_list = books_ori.take(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
for i in books_list:
    print(i)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{'asin': 'B00128EFK0', 'category': 'Religion & Spirituality', 'title': "Approaching the Holy-A Season in God's House", 'author': 'Dennis G. Crump', 'rank': '4646191', 'recommendation': []}
{'asin': 'B00129LKMK', 'category': 'Arts & Photography', 'title': "Bellini's I PURITANI Opera Journeys Mini Guide (Opera Journeys Mini Guide Series) eBook", 'author': 'Burton D. Fisher', 'rank': '3489361', 'recommendation': []}
{'asin': 'B0012AXRXO', 'category': 'Mystery, Thriller & Suspense', 'title': 'Snowman', 'author': 'Dave Saari', 'rank': '2283726', 'recommendation': []}
{'asin': 'B0012BKPEW', 'category': 'Literature & Fiction', 'title': 'Crumbs and Crutons', 'author': 'David Greer', 'rank': '4741892', 'recommendation': []}
{'asin': 'B0012BS7UQ', 'category': 'Religion & Spirituality', 'title': 'Fast Track to Victory, A Christian Guidebook', 'author': 'Cheryl Rogers', 'rank': '3182718', 'recommendation': []}
{'asin': 'B0012BRQNK', 'category': 'Arts & Photography', 'title': 'To Roma! To Roma! (It

In [15]:
from pyspark.sql.types import StructType,StructField,StringType
fields =['asin','category','title','author','rank','recommendation']

schema = StructType([StructField(field,StringType(),True) for field in fields])

jsDF = spark.createDataFrame(books_ori,schema)

jsDF.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------------------+--------------------+--------------------+-------+--------------------+
|      asin|            category|               title|              author|   rank|      recommendation|
+----------+--------------------+--------------------+--------------------+-------+--------------------+
|B00128EFK0|Religion & Spirit...|Approaching the H...|     Dennis G. Crump|4646191|                  []|
|B00129LKMK|  Arts & Photography|Bellini's I PURIT...|    Burton D. Fisher|3489361|                  []|
|B0012AXRXO|Mystery, Thriller...|             Snowman|          Dave Saari|2283726|                  []|
|B0012BKPEW|Literature & Fiction|  Crumbs and Crutons|         David Greer|4741892|                  []|
|B0012BS7UQ|Religion & Spirit...|Fast Track to Vic...|       Cheryl Rogers|3182718|                  []|
|B0012BRQNK|  Arts & Photography|To Roma! To Roma!...|        John Galavan|1853702|                  []|
|B0012BTXPE|   Children's eBooks|How to Study Smar...|D

In [16]:
categories = jsDF.select('category').distinct()
categories.show(150, False)
categories.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------------------------+
|category                                   |
+-------------------------------------------+
|Lesbian, Gay, Bisexual & Transgender eBooks|
|Sports                                     |
|Parenting & Relationships                  |
|Politics & Social Sciences                 |
|Biographies & Memoirs                      |
|Engineering & Transportation               |
|Romance                                    |
|Cookbooks, Food & Wine                     |
|45 minutes (22-32 pages)                   |
|Nonfiction                                 |
|30 minutes (12-21 pages)                   |
|Education & Teaching                       |
|History                                    |
|Kindle Keyboard                            |
|Self-Help                                  |
|Industry Focus                             |
|Kindle (5th Generation)                    |
|                                           |
|Health, Fitness & Dieting        

In [33]:
profDF.filter("asin == 'B0064BV1YI'").collect(False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Row(asin='B0064BV1YI', price='', imUrl='http://ecx.images-amazon.com/images/I/41cWEO1MXjL._BO2,204,203,200_PIsitb-sticker-v3-big,TopRight,0,-55_SX278_SY278_PIkin4,BottomRight,1,22_AA300_SH20_OU01_.jpg', title_prof='', author='', cate='Foreign Languages', description='', related='[B006NPCZ5Y, B0089CD3JQ, B006U44AU6, B005JHHS90, B004NIFXBG, B00IS8RYI2, B00C44CFI0, B009DSD3GI, B009DS1VNA, B00E8FNB9K]')]

In [39]:
left_join = profDF.join(jsDF, profDF.asin == jsDF.asin,how='left') # Could also use 'left_outer'
left_join = left_join.toDF('asin', 'price', 'imUrl', 'title_prof', 'author_prof', 'cate', 'description', 'related', 'asin2', 'category', 'title', 'author', 'rank', 'recommend')


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+--------------------+----------+-----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-------+--------------------+
|      asin|price|               imUrl|title_prof|author_prof|                cate|         description|             related|     asin2|            category|               title|              author|   rank|           recommend|
+----------+-----+--------------------+----------+-----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-------+--------------------+
|B000F83STC|     |http://g-ecx.imag...|          |           |Literature & Fiction|                    |                  []|      null|                null|                null|                null|   null|                null|
|B000FBFL4E|     |http://ecx.images...|          |           |Professional & Te...|T

In [41]:
left_join = left_join.na.fill("")
left_join.show(200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+--------------------+----------+-----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-------+--------------------+
|      asin|price|               imUrl|title_prof|author_prof|                cate|         description|             related|     asin2|            category|               title|              author|   rank|           recommend|
+----------+-----+--------------------+----------+-----------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+-------+--------------------+
|B000F83STC|     |http://g-ecx.imag...|          |           |Literature & Fiction|                    |                  []|          |                    |                    |                    |       |                    |
|B000FBFL4E|     |http://ecx.images...|          |           |Professional & Te...|T

In [40]:
left_join.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

434702

In [44]:
from pyspark.sql import functions as f
left_join = left_join.withColumn('title_final', f.when(f.col('title_prof') != "", f.col('title_prof')).otherwise(f.col('title')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
left_join = left_join.withColumn('author_final', f.when(f.col('author_prof') != "", f.col('author_prof')).otherwise(f.col('author')))
left_join = left_join.withColumn('category_final', f.when(f.col('cate') != "", f.col('cate')).otherwise(f.col('category')))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [51]:
left_join = left_join.drop('title_prof', 'author_prof', 'cate', 'asin2', 'category', 'title', 'author', 'recommend')
left_join = left_join.toDF('asin','price', 'imUrl', 'description', 'related','rank','title','author', 'category')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
left_join.show(200)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-----+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+
|      asin|price|               imUrl|         description|             related|   rank|               title|              author|            category|
+----------+-----+--------------------+--------------------+--------------------+-------+--------------------+--------------------+--------------------+
|B000F83STC|     |http://g-ecx.imag...|                    |                  []|       |                    |                    |Literature & Fiction|
|B000FBFL4E|     |http://ecx.images...|The terms and inf...|                  []|       |                    |                    |Professional & Te...|
|B000FBFP5O| 1.95|http://ecx.images...|                    |                  []|       |                    |                    |Science Fiction &...|
|B000FC0YZO| 9.99|http://ecx.images...|                    |[B007BZC13G, B001...| 

In [55]:
left_join.filter("title != ''").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

190899

In [56]:
df_json = left_join.toJSON()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
df_json.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['{"asin":"B000JMKMZM","price":"","imUrl":"http://ecx.images-amazon.com/images/I/41foZmhNNrL._BO2,204,203,200_PIsitb-sticker-v3-big,TopRight,0,-55_SX278_SY278_PIkin4,BottomRight,1,22_AA300_SH20_OU01_.jpg","description":"\\"Steroids has nothing on these guys!Place a bunch of villains in a cellblock and you may have a riot. Now place a bunch of villains with super powers, plus one mighty pissed off innocent hero in the pot and you get Terry L. Vinson\'sDesolation Island.The book begins with our main and lovable character Ben Thomason reminiscing with an old friend about the good ol\' days when ass kicking and chasing villains was an \'honorable\' job to have. As the conversation progresses, you experience his views on how the system has changed...but more than that you find out Ben is hiding out, framed for a murder he didn\'t commit.Immediately within several pages, a reader is thrust into a super fight...and I mean super strength fight with an unsuspecting foe for Ben.Defeated, enraged

In [61]:
df_json.saveAsTextFile("s3n://analytics-50043/output/kindle_metadata_cleaned.json")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…